In [28]:
# import packages
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
import numpy as np
import glob
import rioxarray as rxr
import xarray as xr

In [11]:
# create function to clip raster extent and save to file without reprojecting
def clip_raster_no_reprojection(in_path_ras, in_path_vec, out_path):

    # Load vector data
    vector = gpd.read_file(in_path_vec)

    # Open the raster file
    with rasterio.open(in_path_ras) as src:
        # Reproject the vector to match the raster CRS
        vector = vector.to_crs(src.crs)

        # Convert vector geometry to GeoJSON format
        geometries = [geom for geom in vector.geometry]

        # Clip the raster
        clipped_raster, clipped_transform = mask(src, geometries, crop=True)

        # Update the metadata
        out_meta = src.meta.copy()

        # Check if NoData is defined
        if src.nodata is None:  # Apply changes only if NoData is not defined
        # Set a NoData value (e.g., -9999)
            clipped_raster[clipped_raster == 0] = -9999  # Replace 0 if needed
            out_meta.update({"nodata": -9999})

        out_meta.update({
            "driver": "GTiff",
            "height": clipped_raster.shape[1],
            "width": clipped_raster.shape[2],
            "transform": clipped_transform
        })

        # Save the clipped raster
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(clipped_raster)

    print(f"Clipped raster saved to {out_path} in original CRS {src.crs}")

In [25]:
# define print raster function to help compare reprojections
def print_raster(raster):
    print(
        f"shape: {raster.rio.shape}\n"
        f"resolution: {raster.rio.resolution()}\n"
        f"bounds: {raster.rio.bounds()}\n"
        f"sum: {raster.sum().item()}\n"
        f"CRS: {raster.rio.crs}\n"
        f"NoDataValue: {raster.rio.nodata}\n"
        f"NoDataOriginalValues: {raster.rio.encoded_nodata}\n"
    )

In [12]:
# set file paths 
# spawn 2010 datasets in paths
agb = './carbon_stock_data/cstock_spawn/agb_2010.tif'
agb_uncertainty = './carbon_stock_data/cstock_spawn/agb_uncertainty_2010.tif'
bgb = './carbon_stock_data/cstock_spawn/bgb_2010.tif'
bgb_uncertainty = './carbon_stock_data/cstock_spawn/bgb_uncertainty_2010.tif'

# spawn 2010 datasets out paths
agb_out = './carbon_stock_data/final_layers_clipped/agb_mg_ha_spawn_2010.tif'
agb_uncertainty_out = './carbon_stock_data/final_layers_clipped/agb_uncertainty_mg_ha_spawn_2010.tif'
bgb_out = './carbon_stock_data/final_layers_clipped/bgb_mg_ha_spawn_2010.tif'
bgb_uncertainty_out = './carbon_stock_data/final_layers_clipped/bgb_uncertainty_mg_ha_spawn_2010.tif'

# sothe datasets in paths
fc = './carbon_stock_data/cstock_canada_sothe/McMaster_WWFCanada_forest_carbon_250m/McMaster_WWFCanada_forest_carbon_250m_kg-m2_version1.0.tif'
fc_uncertainty = './carbon_stock_data/cstock_canada_sothe/McMaster_WWFCanada_forest_carbon_250m/McMaster_WWFCanada_forest_carbon_250m_kg-m2_uncertainty_version1.0.tif'
soc1m = './carbon_stock_data/cstock_canada_sothe/McMaster_WWFCanada_soil_carbon_250m_kg-m2_version3.0/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3.0.tif'
soc1m_uncertainty = './carbon_stock_data/cstock_canada_sothe/McMaster_WWFCanada_soil_carbon_250m_kg-m2_version3.0/McMaster_WWFCanada_soil_carbon1m_uncertainty_250m_kg-m2_version3.0.tif'

# sothe datasets out paths
fc_out = './carbon_stock_data/final_layers_clipped/forest_carbon_kg_m2_sothe.tif'
fc_uncertainty_out = './carbon_stock_data/final_layers_clipped/forest_carbon_uncertainty_kg_m2_sothe.tif'
soc1m_out = './carbon_stock_data/final_layers_clipped/soc_0_1m_kg_m2_sothe.tif'
soc1m_uncertainty_out = './carbon_stock_data/final_layers_clipped/soc_uncertainty_0_1m_kg_m2_sothe.tif'

# y2y boundary
y2y = './study_area/Y2Y_RegionBoundary_Final2013/Y2Y_RegionBoundary.shp'


In [14]:
# clip rasters
# spawn
clip_raster_no_reprojection(agb, y2y, agb_out)
clip_raster_no_reprojection(agb_uncertainty, y2y, agb_uncertainty_out)
clip_raster_no_reprojection(bgb, y2y, bgb_out)
clip_raster_no_reprojection(bgb_uncertainty, y2y, bgb_uncertainty_out)

# sothe
clip_raster_no_reprojection(fc, y2y, fc_out)
clip_raster_no_reprojection(fc_uncertainty, y2y, fc_uncertainty_out)
clip_raster_no_reprojection(soc1m, y2y, soc1m_out)
clip_raster_no_reprojection(soc1m_uncertainty, y2y, soc1m_uncertainty_out)


Clipped raster saved to ./carbon_stock_data/final_layers_clipped/agb_mg_ha_spawn_2010.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/agb_uncertainty_mg_ha_spawn_2010.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/bgb_mg_ha_spawn_2010.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/bgb_uncertainty_mg_ha_spawn_2010.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/forest_carbon_kg_m2_sothe.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/forest_carbon_uncertainty_kg_m2_sothe.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_0_1m_kg_m2_sothe.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_uncertainty_0_1m_kg_m2_sothe.tif in original CRS EPSG:4326


In [15]:
# list file names of soilgrids products
soc = './carbon_stock_data/soc_soilgrids/ocs_0-30cm_mean.tif'
soc_out = './carbon_stock_data/final_layers_clipped/soc_mean_0_30cm_t_ha_soilgrids.tif'

soc_05 = './carbon_stock_data/soc_soilgrids/ocs_0-30cm_Q0.05.tif'
soc_05_out = './carbon_stock_data/final_layers_clipped/soc_q05_0_30cm_t_ha_soilgrids.tif'

soc_95 = './carbon_stock_data/soc_soilgrids/ocs_0-30cm_Q0.95.tif'
soc_95_out = './carbon_stock_data/final_layers_clipped/soc_q95_0_30cm_t_ha_soilgrids.tif'

soc_uncertainty = './carbon_stock_data/soc_soilgrids/ocs_0-30cm_uncertainty.tif'
soc_uncertainty_out = './carbon_stock_data/final_layers_clipped/soc_uncertainty_0_30cm_t_ha_soilgrids.tif'

In [16]:
# # clip rasters
# clip_raster(soc, y2y, soc_out, target_crs)
# clip_raster(soc_05, y2y, soc_05_out, target_crs)
# clip_raster(soc_95, y2y, soc_95_out, target_crs)
# clip_raster(soc_uncertainty, y2y, soc_uncertainty_out, target_crs)

clip_raster_no_reprojection(soc, y2y, soc_out)
clip_raster_no_reprojection(soc_05, y2y, soc_05_out)
clip_raster_no_reprojection(soc_95, y2y, soc_95_out)
clip_raster_no_reprojection(soc_uncertainty, y2y, soc_uncertainty_out)

Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_mean_0_30cm_t_ha_soilgrids.tif in original CRS PROJCS["Interrupted_Goode_Homolosine",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Interrupted_Goode_Homolosine"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_q05_0_30cm_t_ha_soilgrids.tif in original CRS PROJCS["Interrupted_Goode_Homolosine",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Interrupted_Goode_Homolosine"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_q9

In [17]:
# moving on to the soc maps from openlandmap
# load file paths
soc_0_10 = './carbon_stock_data/soc_openlandmap/sol_organic.carbon.stock_msa.kgm2_m_250m_b0..10cm_1950..2017_v0.2.tif'
soc_10_30 = './carbon_stock_data/soc_openlandmap/sol_organic.carbon.stock_msa.kgm2_m_250m_b10..30cm_1950..2017_v0.2.tif'
soc_30_60 = './carbon_stock_data/soc_openlandmap/sol_organic.carbon.stock_msa.kgm2_m_250m_b30..60cm_1950..2017_v0.2.tif'
soc_60_100 = './carbon_stock_data/soc_openlandmap/sol_organic.carbon.stock_msa.kgm2_m_250m_b60..100cm_1950..2017_v0.2.tif'

In [23]:
# list all layers
soc_dir = [soc_0_10, soc_10_30, soc_30_60, soc_60_100]

# already clipped and deleted original

# # first clip layers to y2y extent
# for rast in soc_dir:
#     clip_raster_no_reprojection(rast, y2y, rast.replace('.tif', '_clipped.tif'))

In [ ]:
# now add all layers together
for i in range(0, len(soc_dir)):
    if i == 0:
        soc = xr.open_dataarray(soc_dir[i].replace('.tif', '_clipped.tif'))
    else:
        soc += xr.open_dataarray(soc_dir[i].replace('.tif', '_clipped.tif'))

print_raster(soc)

shape: (12274, 15713)
resolution: (0.002083333, -0.002083333)
bounds: (-141.035422901, 41.751257299, -108.30001147200001, 67.322086541)
sum: 1455630080.0
CRS: EPSG:4326
NoDataValue: nan
NoDataOriginalValues: -32768.0



In [27]:
# write to disk
soc.rio.to_raster('./carbon_stock_data/final_layers_clipped/soc_0_1m_kg_m2_openlandmap.tif')

In [29]:
# list layers from 30-100
soc_dir_30_100 = [soc_30_60, soc_60_100]

# now add layers together
for i in range(0, len(soc_dir_30_100)):
    if i == 0:
        soc_30_100 = xr.open_dataarray(soc_dir[i].replace('.tif', '_clipped.tif'))
    else:
        soc_30_100 += xr.open_dataarray(soc_dir[i].replace('.tif', '_clipped.tif'))

print_raster(soc_30_100)

shape: (12274, 15713)
resolution: (0.002083333, -0.002083333)
bounds: (-141.035422901, 41.751257299, -108.30001147200001, 67.322086541)
sum: 707438976.0
CRS: EPSG:4326
NoDataValue: nan
NoDataOriginalValues: -32768.0



In [30]:
# write to disk
soc_30_100.rio.to_raster('./carbon_stock_data/final_layers_clipped/soc_30_100cm_kg_m2_openlandmap.tif')